## **Installing LibAUC**


In [1]:
  ! pip install medmnist
  ! pip install libauc==1.2.0
  ! pip install tensorboardX
  ! pip install acsconv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116952 sha256=ec4daf033dcb8dd939582415bd447ab49c20cff428de964c35d363937197a43f
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 4.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done

# **Importing Libraries**


In [2]:
import os
import time
from PIL import Image
import numpy as np
import random

from libauc.losses import AUCMLoss
from libauc.optimizers import PESG
from libauc.models import resnet20 as ResNet20
from libauc.models import resnet18 as ResNet18
from libauc.utils import ImbalancedDataGenerator
from libauc.sampler import DualSampler
from libauc.metrics import auc_roc_score

import medmnist
from medmnist import VesselMNIST3D
from medmnist import INFO, Evaluator
import torch.utils.data as data

import torch 
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from acsconv.converters import Conv3dConverter


import warnings
warnings.filterwarnings('ignore')

The ``converters`` are currently experimental. It may not support operations including (but not limited to) Functions in ``torch.nn.functional`` that involved data dimension


## **Reproducibility**



In [3]:
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
class Transform3D:

    def __init__(self, mul=None):
        self.mul = mul

    def __call__(self, voxel):
   
        if self.mul == '0.5':
            voxel = voxel * 0.5

        elif self.mul == 'random':
            voxel = voxel * np.random.uniform()
        
        return voxel.astype(np.float32)

# **Paramaters**

In [18]:
# # HyperParameters

lr = 0.01
margin = 1.0
epoch_decay = 0.03
weight_decay = 0.0001
BATCH_SIZE=128
momentum=0.9
shape_transform = True
total_epochs = 30 
decay_epochs = [50, 75]

# **Loading datasets**

In [19]:
# Load VesselMNIST3D
data_flag = 'vesselmnist3d'
download = True
as_rgb = True
info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])
DataClass = getattr(medmnist, info['python_class'])
info

{'python_class': 'VesselMNIST3D',
 'description': 'The VesselMNIST3D is based on an open-access 3D intracranial aneurysm dataset, IntrA, containing 103 3D models (meshes) of entire brain vessels collected by reconstructing MRA images. 1,694 healthy vessel segments and 215 aneurysm segments are generated automatically from the complete models. We fix the non-watertight mesh with PyMeshFix and voxelize the watertight mesh with trimesh into 28×28×28 voxels. We split the source dataset with a ratio of 7:1:2 into training, validation and test set.',
 'url': 'https://zenodo.org/record/6496656/files/vesselmnist3d.npz?download=1',
 'MD5': '2ba5b80617d705141f3f85627108fce8',
 'task': 'binary-class',
 'label': {'0': 'vessel', '1': 'aneurysm'},
 'n_channels': 1,
 'n_samples': {'train': 1335, 'val': 192, 'test': 382},
 'license': 'CC BY 4.0'}

In [20]:
# Transformations
train_transform = Transform3D(mul='random') if shape_transform else Transform3D()
eval_transform = Transform3D(mul='0.5') if shape_transform else Transform3D()

# Define Datasets
train_dataset = DataClass(split='train', transform=train_transform, download=download, as_rgb=as_rgb)
train_dataset_at_eval = DataClass(split='train', transform=eval_transform, download=download, as_rgb=as_rgb)
val_dataset = DataClass(split='val', transform=eval_transform, download=download, as_rgb=as_rgb)
test_dataset = DataClass(split='test', transform=eval_transform, download=download, as_rgb=as_rgb)

# Load data
trainloader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
trainloader_eval = data.DataLoader(dataset=train_dataset_at_eval, batch_size=2*BATCH_SIZE, shuffle=False)
val_loader = data.DataLoader(dataset=val_dataset, batch_size=2*BATCH_SIZE, shuffle=False)
testloader = data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)


Using downloaded and verified file: /root/.medmnist/vesselmnist3d.npz
Using downloaded and verified file: /root/.medmnist/vesselmnist3d.npz
Using downloaded and verified file: /root/.medmnist/vesselmnist3d.npz
Using downloaded and verified file: /root/.medmnist/vesselmnist3d.npz


# **Creating models & AUC Optimizer**

In [21]:
# You can include sigmoid/l2 activations on model's outputs before computing loss

model = ResNet18(pretrained=False, last_activation=None) 
model.conv1 = nn.Conv2d(3, 64, kernel_size=3,stride=1, padding=1, bias=False)
model = Conv3dConverter(model)
model = model.cuda()
loss_fn = AUCMLoss()
optimizer = PESG(model, 
                 loss_fn=loss_fn,
                 lr=lr, 
                 momentum=momentum,
                 margin=margin, 
                 epoch_decay=epoch_decay, 
                 weight_decay=weight_decay)

# **Training**

In [22]:
print ('Start Training')
print ('-'*30)

best_val_auc = 0
best_model = model

train_log = []
test_log = []

for epoch in range(total_epochs):
     if epoch in decay_epochs:
         optimizer.update_regularizer(decay_factor=10) # decrease learning rate by 10x & update regularizer
   
     # TRAINING   
     train_loss = []
     model.train()    
     for train_data, train_targets in trainloader:
         train_data, train_targets  = train_data.cuda(), train_targets.cuda()
         y_pred = model(train_data)
         y_pred = torch.sigmoid(y_pred)
         loss = loss_fn(y_pred, train_targets)
         optimizer.zero_grad()
         loss.backward()
         optimizer.step()
         train_loss.append(loss.item())
    
     epoch_loss = np.mean(train_loss)

     # VALIDATION
     with torch.no_grad():
        model.eval()
        val_pred_list = []
        val_true_list = []
        for val_data, val_targets in val_loader:
            val_data  = val_data.cuda()
            val_pred = model(val_data)
            val_pred_list.append(val_pred.cpu().detach().numpy())
            val_true_list.append(val_targets.numpy())
        val_true = np.concatenate(val_true_list)
        val_pred = np.concatenate(val_pred_list)
        val_auc = auc_roc_score(val_true, val_pred)

        val_pred_binary = (val_pred > 0.5).astype(int)
        val_accuracy = accuracy_score(val_true, val_pred_binary)
          
        if best_val_auc < val_auc:
          best_val_auc = val_auc
          best_model = model

        train_log.append(val_auc)    
       

     print("epoch: %s, epoch_loss: %.4f, val_auc: %.4f, lr: %.4f, best_val_auc: %.4f"%(epoch, epoch_loss, val_auc, optimizer.lr, best_val_auc))    

Start Training
------------------------------
epoch: 0, epoch_loss: 0.0374, val_auc: 0.4072, lr: 0.0100, best_val_auc: 0.4072
epoch: 1, epoch_loss: 0.0563, val_auc: 0.4112, lr: 0.0100, best_val_auc: 0.4112
epoch: 2, epoch_loss: 0.0702, val_auc: 0.5556, lr: 0.0100, best_val_auc: 0.5556
epoch: 3, epoch_loss: 0.0809, val_auc: 0.6259, lr: 0.0100, best_val_auc: 0.6259
epoch: 4, epoch_loss: 0.0845, val_auc: 0.7072, lr: 0.0100, best_val_auc: 0.7072
epoch: 5, epoch_loss: 0.0887, val_auc: 0.7912, lr: 0.0100, best_val_auc: 0.7912
epoch: 6, epoch_loss: 0.0907, val_auc: 0.8120, lr: 0.0100, best_val_auc: 0.8120
epoch: 7, epoch_loss: 0.0882, val_auc: 0.8128, lr: 0.0100, best_val_auc: 0.8128
epoch: 8, epoch_loss: 0.0857, val_auc: 0.8155, lr: 0.0100, best_val_auc: 0.8155
epoch: 9, epoch_loss: 0.0830, val_auc: 0.8265, lr: 0.0100, best_val_auc: 0.8265
epoch: 10, epoch_loss: 0.0774, val_auc: 0.8382, lr: 0.0100, best_val_auc: 0.8382
epoch: 11, epoch_loss: 0.0791, val_auc: 0.8449, lr: 0.0100, best_val_auc:

In [31]:
dataflag = 'vesselmnist3d'

# **Testing**

In [28]:
# Evaluation on Test data

test_pred_list = []
test_true_list = [] 
for test_data, test_targets in testloader:
    test_data  = test_data.cuda()
    test_pred = best_model(test_data)
    test_pred_list.append(test_pred.cpu().detach().numpy())
    test_true_list.append(test_targets.numpy())
test_true = np.concatenate(test_true_list)
test_pred = np.concatenate(test_pred_list)
test_auc =  auc_roc_score(test_true, test_pred) 

test_pred_binary = (test_pred > 0.5).astype(int)
test_accuracy = accuracy_score(test_true, test_pred_binary)

# print results
print("Test AUC: %.4f, Test Accuracy: %.4f"%(test_auc, test_accuracy))      

Test AUC: 0.8930, Test Accuracy: 0.9136


In [33]:
# Saving the best model
state = {
    'net': best_model.state_dict(),
}

output_root = os.path.join('./output', 'vesselmnist3d')
if not os.path.exists(output_root):
    os.makedirs(output_root)

filename = 'vesselmnist3d' + '_auc_' + str(round(test_auc,4)) + '_model.pth'
path = os.path.join(output_root, filename)
torch.save(state, path)

In [29]:
# Eval function similar to demo file

def evaluate(net, test_loader):
    # Testing AUC
    score_list = list()
    label_list = list()
    for tmp_data, tmp_label in test_loader:
        # tmp_data, tmp_label, tmp_idx = data
        tmp_data, tmp_label = tmp_data.cuda(), tmp_label.cuda()
#         tmp_data = tmp_data.expand(-1, 3, -1, -1)        
        tmp_score = net(tmp_data).detach().clone().cpu()
        score_list.append(tmp_score)
        label_list.append(tmp_label.cpu())
    test_label = torch.cat(label_list)
    test_score = torch.cat(score_list)
                   
    test_auc = metrics.roc_auc_score(test_label, test_score)                   
    print("Test: %.4f"%test_auc, flush=True)

evaluate(best_model, testloader)

Test: 0.8930
